In [3]:
def recommendation_finale(tconst):
    import pandas as pd
    from sklearn.neighbors import NearestNeighbors
    from sklearn.preprocessing import MinMaxScaler

    # ===============================
    # Chargement des données
    # ===============================
    df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

    # ===============================
    # Définition des colonnes pour les genres et les pays
    # ===============================
    colonnes_genre = [
        'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
        'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
        'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
        'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
    ]
    colonnes_pays = [
        'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
        'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
        'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
        'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
        'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
        'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
    ]

    # ===============================
    # Extraction des genres et pays du film sélectionné
    # ===============================
    df_selection = df_ml[df_ml['tconst'] == tconst]
    genre = [col for col in df_selection.columns if col in colonnes_genre and df_selection[col].iloc[0] == True]
    pays = [col for col in df_selection.columns if col in colonnes_pays and df_selection[col].iloc[0] == True]

    # ===============================
    # Normalisation des colonnes numériques
    # ===============================
    df_ml_num = df_ml.select_dtypes('number')  # Colonnes numériques
    df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])  # Colonnes non numériques
    scaler = MinMaxScaler()
    df_ml_num_SN = pd.DataFrame(scaler.fit_transform(df_ml_num), columns=df_ml_num.columns, index=df_ml.index)
    df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)  # Fusion des colonnes normalisées et catégoriques

    # ===============================
    # Sélection des caractéristiques pour le modèle
    # ===============================
    drop_columns = ['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating'] + colonnes_genre + colonnes_pays
    caracteristiques = df_ml_encoded.columns.drop(drop_columns)

    # ===============================
    # Filtrage des films avec une note supérieure ou égale à 0,7
    # ===============================
    bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]

    # ===============================
    # Création des deux sous-ensembles
    # ===============================
    # Films partageant le même genre et pays
    df_genre_country = bons_films[bons_films[genre].any(axis=1) & bons_films[pays].any(axis=1)]

    # Films partageant le même genre mais excluant les pays
    df_genre_no_country = bons_films[bons_films[genre].any(axis=1) & ~bons_films[pays].any(axis=1)]

    # ===============================
    # Fonction de recommandation avec Nearest Neighbors
    # ===============================
    def get_recommendations(data, tconst_features):
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(data[caracteristiques])  # Entraînement du modèle
        distances, indices = model.kneighbors(tconst_features)  # Recherche des voisins
        return distances, indices

    # ===============================
    # Extraction des caractéristiques du film sélectionné
    # ===============================
    caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst][caracteristiques]

    # ===============================
    # Recommandations pour les films avec genres et pays
    # ===============================
    distances1, indices1 = get_recommendations(df_genre_country, caract_film)
    selection1 = (
        df_genre_country.iloc[indices1[0][1:6]]['tconst'].reset_index(drop=True)
        if caract_film['notes'].values[0] > 0.7
        else df_genre_country.iloc[indices1[0][0:5]]['tconst'].reset_index(drop=True)
    )

    # ===============================
    # Recommandations pour les films avec genres mais sans pays
    # ===============================
    distances2, indices2 = get_recommendations(df_genre_no_country, caract_film)
    selection2 = (
        df_genre_no_country.iloc[indices2[0][1:6]]['tconst'].reset_index(drop=True)
        if caract_film['notes'].values[0] > 0.7
        else df_genre_no_country.iloc[indices2[0][0:5]]['tconst'].reset_index(drop=True)
    )

    # ===============================
    # Retour des deux DataFrames
    # ===============================
    return selection1, selection2
